# Exercise 1: Evolution Strategy

## Import libraries

In [1]:
import numpy as np

np.set_printoptions(precision=4, suppress=True) # for better printing

# ES ($\mu$,$\lambda$) algorithm
In this exercise, we will use Evolution Strategies ($\mu$,$\lambda$) as explained in the lecture. The algorithm was introduced with the following steps:

1. Set up population size $\lambda$, number of parents $\mu$, mutation step size $\sigma$
2. Set up solution guess, e.g, $n\times 1$ vector $m$
3. Generate $\lambda$ offspring from $m$
4. Evaluate $\lambda$ offspring
5. Select $\mu$ parents with Truncated Rank Selection
6. Update population $m$ using fitness-weighted values of $\mu$ parents

> Go to 3. until convergence or maximum number of generations

In the following, we will implement the algorithm. Step 4. is problem specific and already prepared. We will select the parameters of Step 1. at the end of the exercise for each problem.

## Step 2: Initialise $m$

-------
### **Assignment**
Implement the `initialise_mean`-function. Initialise the mean by using a random uniform distribution. The mean vector should have the same size as the number of parameters specified. Sample from a uniform distribution in range between -4.0 and 4.0 for the initial mean.

### **Hint**
To sample from a uniform distribution with NumPy, you can use `np.random.uniform`. Check the NumPy [documentation](https://numpy.org/doc/stable/reference/random/generated/numpy.random.uniform.html) for further details. 

-------

In [2]:
def initialise_mean(num_parameters):
    mean_vector = ...
    return mean_vector

In [3]:
# Solution
def initialise_mean(num_parameters):
    mean_vector = np.random.uniform(low=-4.0, high=4.0, size=(num_parameters,))
    return mean_vector

In [4]:
# Test your function:
num_parameters = 5
mean_vector = initialise_mean(num_parameters=num_parameters)

print(f"Mean shape: {mean_vector.shape}\n\nMean parameters:\n{mean_vector}")
assert len(mean_vector) == num_parameters

Mean shape: (5,)

Mean parameters:
[-1.5216 -0.497  -1.075   2.0879 -1.1956]


## Step 3: Generate $\lambda$ offspring from $m$

We introduce the following notation for storing the parameters of our population. The population parameters are stored in a 2D array with dimension (population_size, parameter_size). That means that each row corresponds to the parameters of one population member.

|   | Parameter 1   | Parameter 2   | $\cdots$   | Parameter n   |
|:---:|:---:|:---:|:---:|:---:|
| Candidate 1   | $\dots$  | $\dots$  |  $\dots$ |  $\dots$ |
| Candidate 2  | $\dots$  |  $\dots$ | $\dots$  |  $\dots$ |
| $\vdots$  | $\dots$ | $\dots$  | $\dots$  | $\dots$  |
| Candidate m  |  $\dots$ | $\dots$  |  $\dots$ | $\dots$  |

For generating a new offspring the lecture introduced the following formula:

\begin{align}
    x_i = m + \sigma \mathcal{N}_i(0, \mathcal{I}) \qquad \text{for} \quad 0 < i \le \lambda
\end{align}

---
### **Assignment**
Implement the `generate_offspring`-function with the formula given from the lecture to generate the population as specified above.

### **Hint**
You can generate multivariate (normal) Gaussian distributions with `np.random.normal`. Check the NumPy [documentation](https://numpy.org/doc/stable/reference/random/generated/numpy.random.normal.html) for further details.

---

In [5]:
def generate_mutated_offspring(population_size, mean_vector, mutation_sigma):
    # Duplicate mean candidate along y-dim
    population = ...
    
    # Compute multivariate Gaussian noise
    mutation = ...
    
    # Compute offspring
    mutated_population = ...
    
    return mutated_population

In [6]:
# Solution
def generate_mutated_offspring(population_size, mean_vector, mutation_sigma):   
    # Duplicate mean candidate along y-dim
    population = np.tile(mean_vector,(population_size, 1))
    
    # Compute multivariate Gaussian noise
    num_parameters = len(mean_vector)
    perturbation = np.random.normal(
        loc=0.0, 
        scale=1.0, 
        size=(population_size, num_parameters)
    )
    
    # Compute offspring
    mutated_population = population + mutation_sigma * perturbation
    
    return mutated_population

In [7]:
# Test your function:
population_size = 8
mean_vector = np.array([1.0, 2.0, 3.0, 4.0, 5.0])
mutation_sigma = 0.3

population = generate_mutated_offspring(population_size, mean_vector, mutation_sigma)
print(f"Population shape: {population.shape}")
print(f"Population params:\n{population}")

Population shape: (8, 5)
Population params:
[[1.186  1.6775 3.3114 3.5812 5.1155]
 [1.3828 2.164  2.7308 3.9822 4.9979]
 [0.5853 2.1903 3.3705 3.7652 4.8059]
 [0.8291 1.6264 2.4349 3.5829 4.8647]
 [0.5648 2.5751 3.4339 3.8382 4.6304]
 [0.5038 2.3324 3.3152 4.8278 4.7963]
 [1.1226 2.2739 3.2444 3.6497 4.9855]
 [1.3785 2.2423 3.4473 3.8725 5.5124]]


## Step 5: Select $\mu $ parents with Truncated Rank Selection

After the population has been evaluated, we receive a fitness vector. Use the fitness vector to sort the population and select the best solutions as the parents of a new generation. The parents_population is represented as a 2D array with shape (num_parents, num_parameters). We use the ES ($\mu$,$\lambda$) Strategy so the offspring is fully replaced after each generation.

|   | Parameter 1   | Parameter 2   | $\cdots$   | Parameter n   |
|:---:|:---:|:---:|:---:|:---:|
| Parent 1   | $\dots$  | $\dots$  |  $\dots$ |  $\dots$ |
| Parent 2  | $\dots$  |  $\dots$ | $\dots$  |  $\dots$ |
| $\vdots$  | $\dots$ | $\dots$  | $\dots$  | $\dots$  |
| Parent $\mu$  |  $\dots$ | $\dots$  |  $\dots$ | $\dots$  |

---
### **Assignment**
Implement the `sort_and_select_parents`-function.
1. Sort the population representation according to the given fitness values.
2. Truncate population members which did perform not well enough.

Return the parents population and their corresponding fitness scores. In the problems we want to solve a higher fitness score is better.

### **Hint**
NumPy provides a handy function for sorting an array callend `np.argsort` which returns the indicies of the array according to the **ascending** value.  
With the indicies you can use **indexing** to sort the array. For further details on [argsort](https://numpy.org/doc/stable/reference/generated/numpy.argsort.html) and on [indexing](https://numpy.org/doc/stable/user/basics.indexing.html), check the documentation. 

---


In [8]:
def sort_and_select_parents(population, fitness, num_parents):
    parent_population = ...
    parent_fitness = ...
    return parent_population, parent_fitness

In [9]:
# Solution
def sort_and_select_parents(population, fitness, num_parents):
    sorted_indices = np.argsort(fitness)[::-1]
    sorted_indices = sorted_indices[0:num_parents]
    
    parent_population = population[sorted_indices]
    parent_fitness = fitness[sorted_indices]
    
    return parent_population, parent_fitness

In [10]:
# Test your function:
population_size = 8
num_parameters = 5
num_parents = 4

population = np.random.normal(loc=1.0, size=(population_size, num_parameters))
fitness = np.random.uniform(size=(population_size,))

parent_population, parent_fitness = sort_and_select_parents(
    population=population, 
    fitness=fitness, 
    num_parents=num_parents
)

print(f"Population:\n{population}")
print(f"Population fitness:\n{fitness}")
print()
print(f"Parent population shape: {parent_population.shape}")
print(f"Parent population:\n{parent_population}")
print(f"Parent fitness:\n{parent_fitness}")

Population:
[[ 1.7618  0.5024  0.8941  0.6923 -0.179 ]
 [ 0.5979 -0.6723  1.7106  0.843   0.5217]
 [ 0.8707  0.5132  0.6181  1.151  -0.75  ]
 [ 1.3689  3.0921  2.3227  2.2209  1.2814]
 [ 2.8655 -1.6937  1.3277 -1.2721  1.8081]
 [ 2.0207  1.488   0.3209  1.0215  0.806 ]
 [ 1.524   2.453   1.8044  1.0059  2.7304]
 [ 0.0439  0.7901  0.4233  0.3031  1.065 ]]
Population fitness:
[0.1864 0.4407 0.3965 0.1007 0.4136 0.0939 0.7815 0.1515]

Parent population shape: (4, 5)
Parent population:
[[ 1.524   2.453   1.8044  1.0059  2.7304]
 [ 0.5979 -0.6723  1.7106  0.843   0.5217]
 [ 2.8655 -1.6937  1.3277 -1.2721  1.8081]
 [ 0.8707  0.5132  0.6181  1.151  -0.75  ]]
Parent fitness:
[0.7815 0.4407 0.4136 0.3965]


## Step 6: Update population $m$ using fitness-weighted values of $\mu$ parents

After selecting the parents population 


---
### **Assignment**
Implement the `update_population_mean`-function.
1. Introduce Fitness-weight the parent fitness.
2. Calculate the mean of all population members with weighted fitness.


### **Hint**
In general, weighted mean is calculated according to this equation:
\begin{align}
\mu = \frac{\sum_{i=1}^{N} w_i \cdot \mathbf{x}_i}{\sum_{i=1}^{N} w_i}
\end{align}

There are multiple ways of weighting the population parameters to the fitness scores. One way is using nested `for`-loops. Another computationally more efficient way is to use NumPy matrix multiplications like `np.outer`, `np.vstack` and `np.multiply`.

---

In [11]:
def update_population_mean(parent_population, parent_fitness):
    # Normalise parent fitness scores
    normed_parents_fitness = ...
    
    # Compute population weighted to the normed fitness scores  
    weighted_parents_population = ...
    
    # Calculate the sum of weighted parents population
    updated_mean_vector = ...
    
    return updated_mean_vector

In [12]:
# Solution: Simple
def update_population_mean(parent_population):    
    # Calculate mean of population along y-axis
    updated_mean_vector = np.mean(parent_population, axis=0)
    
    return updated_mean_vector

In [13]:
# Solution: Fitness-weighted
def update_population_mean(parent_population, parent_fitness):
    # Normalise parent fitness scores
    normed_parents_fitness = parent_fitness / np.sum(parent_fitness)
    
    # Compute population weighted to the normed fitness scores    
    weight = np.outer(normed_parents_fitness, np.ones((1, parent_population.shape[1])))
    weighted_parents_population = np.multiply(parent_population, weight) #hadamard product
    
    # Calculate mean of weighted parents population along y-axis
    updated_mean_vector = np.sum(weighted_parents_population, axis=0)
    
    return updated_mean_vector

In [14]:
# Test your function:
np.set_printoptions(precision=4, suppress=True)

num_parameters = 5
num_parents = 4

parent_population = np.random.normal(loc=1.0, scale=0.5, size=(num_parents, num_parameters))
parent_fitness = np.random.uniform(size=(num_parents,))

mean_vector = update_population_mean(parent_population, parent_fitness)

print("====== Small differences ======")
print(f"Best candidate:\n{parent_population[0,:]}")
print(f"Fitness-weighted mean:\n{mean_vector}")
print(f"Population mean unweighted:\n{np.mean(population, axis=0)}")
print()

# Assume one solution clearly outperforms the others.
parent_fitness[0] = 100.0
mean_vector = update_population_mean(parent_population, parent_fitness)

print("====== Large differences ======")
print(f"Best candidate:\n{parent_population[0,:]}")
print(f"Fitness-weighted mean:\n{mean_vector}")
print(f"Parents population mean unweighted:\n{np.mean(parent_population, axis=0)}")

====== Small differences ======
Best candidate:
[1.0159 0.82   1.7515 0.8295 0.8279]
Fitness-weighted mean:
[0.9943 1.2069 1.0914 0.9144 0.3854]
Population mean unweighted:
[1.3817 0.8091 1.1777 0.7457 0.9104]

====== Large differences ======
Best candidate:
[1.0159 0.82   1.7515 0.8295 0.8279]
Fitness-weighted mean:
[1.0156 0.8254 1.7423 0.8307 0.8217]
Parents population mean unweighted:
[1.0479 1.211  1.2488 0.8608 0.4893]
